In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [49]:
df = pd.read_csv('HR_Analytics_한글컬럼.csv')

In [50]:
df['퇴사여부'] = df['퇴사여부'].map({'Yes': 1, 'No': 0})
df['초과근무여부'] = df['초과근무여부'].map({'Yes': 1, 'No': 0})
df['성별'] = df['성별'].map({'Male': 1, 'Female': 0})

In [51]:
df = pd.get_dummies(df, drop_first=True)

In [52]:
X = df.drop('퇴사여부', axis=1)
y = df['퇴사여부']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [76]:
# 불균형 데이터 처리 (퇴사 안 한 사람이 훨씬 많으므로, 퇴사자 예측의 중요도를 가중치로 줌)
#ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1) * 2.0  

In [77]:
model = XGBClassifier(
    n_estimators=200,         # 나무(Tree)를 100개 만듦
    learning_rate=0.05,        # 학습 속도
    max_depth=3,              # 나무의 깊이 (너무 깊으면 과적합 됨)
    min_child_weight=3,
    scale_pos_weight=ratio,   # 불균형 가중치 적용
    random_state=42
)

In [78]:
model.fit(X_train, y_train)
print("✅ 모델 학습 완료!")

✅ 모델 학습 완료!


In [84]:
y_pred = model.predict(X_test)
print("\n--- 📊 모델 평가 결과 ---")
print(f"정확도(Accuracy): {accuracy_score(y_test, y_pred):.2f}")
print("\n상세 성적표:")
print(classification_report(y_test, y_pred))


--- 📊 모델 평가 결과 ---
정확도(Accuracy): 0.79

상세 성적표:
              precision    recall  f1-score   support

           0       0.94      0.80      0.86       248
           1       0.42      0.75      0.54        48

    accuracy                           0.79       296
   macro avg       0.68      0.77      0.70       296
weighted avg       0.86      0.79      0.81       296



In [85]:
import os
os.makedirs('models', exist_ok=True) # 폴더가 없으면 생성

joblib.dump(model, 'models/best_model.pkl')
joblib.dump(X_train.columns.tolist(), 'models/feature_names.pkl') 

print("모델 저장 완료!")

모델 저장 완료!


In [87]:
import pickle
with open('models/feature_names.pkl', 'rb') as f:
    b = pickle.load(f)

In [88]:
b

['나이',
 '집과의거리',
 '교육수준',
 '근무환경만족도',
 '성별',
 '직무몰입도',
 '직급',
 '직무만족도',
 '월급',
 '이전회사근무횟수',
 '초과근무여부',
 '급여인상률',
 '성과평가등급',
 '대인관계만족도',
 '총경력년수',
 '워라밸수준',
 '현회사근속년수',
 '현재직무근무년수',
 '마지막승진후경과년수',
 '출장빈도_Travel_Frequently',
 '출장빈도_Travel_Rarely',
 '부서_영업부',
 '부서_인사부',
 '전공분야_기타',
 '전공분야_마케팅',
 '전공분야_생명과학',
 '전공분야_의학',
 '전공분야_인사',
 '결혼상태_Married',
 '결혼상태_Single',
 '직무분류_생산/관리직',
 '직무분류_연구/관리직',
 '직무분류_연구/기술직',
 '직무분류_영업직',
 '직무분류_인사직',
 '직무분류_전문직']

In [89]:
print(df.columns)

Index(['나이', '퇴사여부', '집과의거리', '교육수준', '근무환경만족도', '성별', '직무몰입도', '직급', '직무만족도',
       '월급', '이전회사근무횟수', '초과근무여부', '급여인상률', '성과평가등급', '대인관계만족도', '총경력년수',
       '워라밸수준', '현회사근속년수', '현재직무근무년수', '마지막승진후경과년수', '출장빈도_Travel_Frequently',
       '출장빈도_Travel_Rarely', '부서_영업부', '부서_인사부', '전공분야_기타', '전공분야_마케팅',
       '전공분야_생명과학', '전공분야_의학', '전공분야_인사', '결혼상태_Married', '결혼상태_Single',
       '직무분류_생산/관리직', '직무분류_연구/관리직', '직무분류_연구/기술직', '직무분류_영업직', '직무분류_인사직',
       '직무분류_전문직'],
      dtype='object')
